In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import re
df = pd.read_parquet('../data/sample.parquet', engine='pyarrow')

In [3]:
df.to_csv('../data/sample.csv', index=True)

In [52]:
df

,time,value,field,robot_id,run_uuid,sensor_type
0,2022-11-23T20:40:00.005Z,821.780800,x,1,8.910096e+18,encoder
1,2022-11-23T20:40:00.017Z,821.821700,x,1,8.910096e+18,encoder
2,2022-11-23T20:40:00.029Z,821.850700,x,1,8.910096e+18,encoder
3,2022-11-23T20:40:00.041Z,821.896400,x,1,8.910096e+18,encoder
4,2022-11-23T20:40:00.053Z,821.957300,x,1,8.910096e+18,encoder
...,...,...,...,...,...,...
1546586,2022-11-23T20:41:17.59Z,-85.692373,fx,1,1.240519e+19,load_cell
1546587,2022-11-23T20:41:17.6Z,-87.231436,fx,1,1.240519e+19,load_cell
1546588,2022-11-23T20:41:17.61Z,-85.649405,fx,1,1.240519e+19,load_cell
1546589,2022-11-23T20:41:17.62Z,-86.430655,fx,1,1.240519e+19,load_cell


In [4]:
def convert_time(x):
    ms_pattern = '\d{4}-[1-9]{2}-[1-9]{2}T\d{2}:\d{2}:\d{2}\.\d*Z'
    sec_pattern = '\d{4}-[1-9]{2}-[1-9]{2}T\d{2}:\d{2}:\d{2}Z'
    if re.match(ms_pattern, x):
        return datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ')
    elif re.match(sec_pattern, x):
        return datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ')
    else:
        return np.nan

There's no NaN values in this dataset, but we can assume if there is any NaN or misformatted time, we should drop that datapoint. 

In [5]:
df['pdtimestamp'] = pd.to_datetime(df['time']) # .apply(convert_time)
print(df.isna().sum().sum())
df.dropna(inplace=True)

0


In [188]:
print(df.keys())
for key in df.keys():
    print(type(df[key][0]))



Index(['time', 'value', 'field', 'robot_id', 'run_uuid', 'sensor_type',
       'pdtimestamp'],
      dtype='object')
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [197]:
print(df.loc[489546,'pdtimestamp'])
print(df[df['time'] == '2022-11-23T20:41:17Z']) # datetime.strptime('2022-11-23T20:41:17.0Z', '%Y-%m-%dT%H:%M:%S.%fZ')])

2022-11-23 20:47:54.335000+00:00
                         time        value field  robot_id      run_uuid  \
826527   2022-11-23T20:41:17Z   481.628401    fx         1  7.582293e+18   
886528   2022-11-23T20:41:17Z    43.796801    fy         1  7.582293e+18   
946529   2022-11-23T20:41:17Z -1724.591704    fz         1  7.582293e+18   
1546527  2022-11-23T20:41:17Z   -84.243155    fx         1  1.240519e+19   

        sensor_type               pdtimestamp  
826527    load_cell 2022-11-23 20:41:17+00:00  
886528    load_cell 2022-11-23 20:41:17+00:00  
946529    load_cell 2022-11-23 20:41:17+00:00  
1546527   load_cell 2022-11-23 20:41:17+00:00  


## 2.1 Understanding the data
There are 4 unique parts in the file.

Encoder values:
min: -771.633000  => origin 
max: 3298.350000  => 3.3m

In [8]:
print(type(df['run_uuid'][0]))
# count number of unique run_uuid
unique_run_uuid = df['run_uuid'].unique()
print(f'{unique_run_uuid[1]:20f}')
print('number of unique run_uuid:', len(unique_run_uuid))


# get average of each sensor type
# by_sensor_type = df.groupby('sensor_type')['value']
# print(by_sensor_type.min(), by_sensor_type.max(), by_sensor_type.mean(), by_sensor_type.std())


# get x, y, z for encoder sensor type
encoder = df[df['sensor_type'] == 'encoder']
print('encoder value range:', encoder['value'].min(), encoder['value'].max(), encoder['value'].mean(), encoder['value'].std())
x = encoder[encoder['field']=='x']
print('encoder x range:', x['value'].min(), x['value'].max(), x['value'].mean(), x['value'].std())
y = encoder[encoder['field']=='y']
print('encoder y range:', y['value'].min(), y['value'].max(), y['value'].mean(), y['value'].std())
z = encoder[encoder['field']=='z']
print('encoder z range:', z['value'].min(), z['value'].max(), z['value'].mean(), z['value'].std())


# get x, y, z for load_cell sensor type
load_cell = df[df['sensor_type'] == 'load_cell']
print('load cell value range:', load_cell['value'].min(), load_cell['value'].max(), load_cell['value'].mean(), load_cell['value'].std())
fx = load_cell[load_cell['field']=='fx']
print('load cell fx range:', fx['value'].min(), fx['value'].max(), fx['value'].mean(), fx['value'].std())
fy = load_cell[load_cell['field']=='fy']
print('load cell fy range:', fy['value'].min(), fy['value'].max(), fy['value'].mean(), fy['value'].std())
fz = load_cell[load_cell['field']=='fz']
print('load cell fz range:', fz['value'].min(), fz['value'].max(), fz['value'].mean(), fz['value'].std())



<class 'numpy.float64'>
7582293080991469568.000000
number of unique run_uuid: 4
encoder value range: -771.633 3298.35 832.5900511217878 1017.6780462582361
encoder x range: 693.1998 3298.35 1766.8628798119391 945.2264532660514
encoder y range: 316.6191 1233.344 761.1501298024627 291.9608832441766
encoder z range: -771.633 -1.7623 -211.72249812725988 291.06528403344475
load cell value range: -1848.4186083984373 875.1750048828125 -186.3552420897327 582.9076181212096
load cell fx range: -1718.656328125 663.8337231445313 -237.26581814432706 511.97422031637404
load cell fy range: -957.0925732421877 875.1750048828125 82.21266780450689 450.50478231733763
load cell fz range: -1848.4186083984373 90.85765502929686 -402.36561879401137 659.7890037699254


In [10]:
# print unique timestamp values
print('number of unique timestamps:', len(df['time'].unique()))

number of unique timestamps: 386359


## 2.2 Timeseries -> Features

In [19]:
# convert time series to x, y, z measurements
# create a new dataframe with column names: | time | fx_1 | fx_2 | fy_1 | fy_2 | fz_1 | fz_2 | x_1 | x_2 | y_1 | y_2 | z_1 | z_2 |

features = pd.DataFrame({
    'run_uuid': pd.Series(dtype=np.float64),
    'time': pd.Series(dtype=str), # df['time'].unique(),
    'fx_1': pd.Series(dtype=np.float64),
    'fx_2': pd.Series(dtype=np.float64),
    'fy_1': pd.Series(dtype=np.float64),
    'fy_2': pd.Series(dtype=np.float64),
    'fz_1': pd.Series(dtype=np.float64),
    'fz_2': pd.Series(dtype=np.float64),
    'x_1': pd.Series(dtype=np.float64),
    'x_2': pd.Series(dtype=np.float64),
    'y_1': pd.Series(dtype=np.float64),
    'y_2': pd.Series(dtype=np.float64),
    'z_1': pd.Series(dtype=np.float64),
    'z_2': pd.Series(dtype=np.float64)
})
features

,run_uuid,time,fx_1,fx_2,fy_1,fy_2,fz_1,fz_2,x_1,x_2,y_1,y_2,z_1,z_2


In [6]:
uuidtime = df.groupby(['run_uuid','pdtimestamp'])
time = df.groupby(['time'])

In [186]:
print(len(uuidtime.groups))
print(len(time.groups))

489547
386359


In [ ]:

            # 'fx_1':row[1]['value'] if row[1]['field'] == 'fx' and row[1]['robot_id'] == 1 else np.nan,
            # 'fx_2':row[1]['value'] if row[1]['field'] == 'fx' and row[1]['robot_id'] == 2 else np.nan,
            # 'fy_1':row[1]['value'] if row[1]['field'] == 'fy' and row[1]['robot_id'] == 1 else np.nan,
            # 'fy_2':row[1]['value'] if row[1]['field'] == 'fy' and row[1]['robot_id'] == 2 else np.nan,
            # 'fz_1':row[1]['value'] if row[1]['field'] == 'fz' and row[1]['robot_id'] == 1 else np.nan,
            # 'fz_2':row[1]['value'] if row[1]['field'] == 'fz' and row[1]['robot_id'] == 2 else np.nan,
            # 'x_1':row[1]['value'] if row[1]['field'] == 'x' and row[1]['robot_id'] == 1 else np.nan,
            # 'x_2':row[1]['value'] if row[1]['field'] == 'x' and row[1]['robot_id'] == 2 else np.nan,
            # 'y_1':row[1]['value'] if row[1]['field'] == 'y' and row[1]['robot_id'] == 1 else np.nan,
            # 'y_2':row[1]['value'] if row[1]['field'] == 'y' and row[1]['robot_id'] == 2 else np.nan,
            # 'z_1':row[1]['value'] if row[1]['field'] == 'z' and row[1]['robot_id'] == 1 else np.nan,
            # 'z_2':row[1]['value'] if row[1]['field'] == 'z' and row[1]['robot_id'] == 2 else np.nan

        # if feature[1]['field'] == 'fx' and feature[1]['robot_id'] == 1:
        #     row['fx_1'] = feature[1]['value']
        # elif feature[1]['field'] == 'fx' and feature[1]['robot_id'] == 2:
        #     row['fx_2'] = feature[1]['value']
        # elif feature[1]['field'] == 'fy' and feature[1]['robot_id'] == 1:
        #     row['fy_1'] = feature[1]['value']
        # elif feature[1]['field'] == 'fy' and feature[1]['robot_id'] == 2:
        #     row['fy_2'] = feature[1]['value']
        # elif feature[1]['field'] == 'fz' and feature[1]['robot_id'] == 1:
        #     row['fz_1'] = feature[1]['value']
        # elif feature[1]['field'] == 'fz' and feature[1]['robot_id'] == 2:
        #     row['fz_2'] = feature[1]['value']
        # elif feature[1]['field'] == 'x' and feature[1]['robot_id'] == 1:
        #     row['x_1'] = feature[1]['value']
        # elif feature[1]['field'] == 'x' and feature[1]['robot_id'] == 2:
        #     row['x_2'] = feature[1]['value']
        # elif feature[1]['field'] == 'y' and feature[1]['robot_id'] == 1:
        #     row['y_1'] = feature[1]['value']
        # elif feature[1]['field'] == 'y' and feature[1]['robot_id'] == 2:
        #     row['y_2'] = feature[1]['value']
        # elif feature[1]['field'] == 'z' and feature[1]['robot_id'] == 1:
        #     row['z_1'] = feature[1]['value']
        # elif feature[1]['field'] == 'z' and feature[1]['robot_id'] == 2:
        #     row['z_2'] = feature[1]['value']

In [7]:
# TODO error checking: 
# assume no duplicate fields for a given timestamp
# TODO benchmark this function, make it faster
# assume time series is sorted by time
rows = []
# indexes = []
for group_index, group_fields in uuidtime:
    # index = pd.Series({'run_uuid':group_index[0], 
    #         'time':group_index[1]})
    row = pd.Series({
            'run_uuid':group_index[0], 
            'pddatetime':group_index[1],
            'fx_1': np.nan,
            'fx_2': np.nan,
            'fy_1': np.nan,
            'fy_2': np.nan,
            'fz_1': np.nan,
            'fz_2': np.nan,
            'x_1':  np.nan,
            'x_2':  np.nan,
            'y_1':  np.nan,
            'y_2':  np.nan,
            'z_1':  np.nan,
            'z_2':  np.nan
        })
    for i, feature in group_fields.iterrows():
        # use field and robot_id to determine which column to populate
        # -> need to check if column name exists
        field_name = feature['field'] + '_' + str(feature['robot_id'])
        # if field_name in row.index:
        row[field_name] = feature['value']
    # indexes.append(index)
    rows.append(row)
    


In [8]:
# indexes = pd.MultiIndex.from_tuples(indexes, names=['run_uuid', 'time'])
# features = pd.DataFrame(rows, index=indexes)
features = pd.DataFrame(rows)
features.sort_values(by='pddatetime', inplace=True)
print(len(features))
print(features.head())

489547
            run_uuid                       pddatetime         fx_1  \
200151  7.582293e+18 2022-11-23 20:40:00.001000+00:00 -1192.046953   
200152  7.582293e+18 2022-11-23 20:40:00.003000+00:00          NaN   
481783  1.240519e+19 2022-11-23 20:40:00.003000+00:00   -88.747061   
392042  8.910096e+18 2022-11-23 20:40:00.005000+00:00          NaN   
0       6.176977e+18 2022-11-23 20:40:00.007000+00:00   176.096381   

              fx_2        fy_1        fy_2         fz_1       fz_2       x_1  \
200151         NaN  716.528276         NaN -1547.340972        NaN       NaN   
200152 -546.669903         NaN  489.207227          NaN  84.484822       NaN   
481783         NaN         NaN         NaN          NaN        NaN       NaN   
392042         NaN         NaN         NaN          NaN        NaN  821.7808   
0              NaN  174.268623         NaN  -258.179417        NaN       NaN   

        x_2       y_1  y_2     z_1  z_2  
200151  NaN       NaN  NaN     NaN  NaN  
200152 

In [229]:
# TODO figure out how to resample the time so that it is evenly spaced
# features['pddatetime'] = features.resample('5L', on='pddatetime')

### Interpolate missing values
A few different ways I thought we could fill in the missing values:
- naive: squish groups of readings together, use the average of timestamp of robot 1 and 2's position measurement 
    - position measurement because they are the most critical 

In [9]:
# how to combine the different measurements into a single row?

# sort by time
# while all fields are not filled, keep taking the next measurement and fill in the fields
# if there are multiple measurements for a given field, take the last one
table_by_uuid = {}
feat_groups = features.groupby(['run_uuid'])
for run_id, run_data in feat_groups:
    run_data.sort_values(by='pddatetime', ascending=True, inplace=True)
    for measurement_name in ['fx_1', 'fx_2', 'fy_1', 'fy_2', 'fz_1', 'fz_2', 'x_1', 'x_2', 'y_1', 'y_2', 'z_1', 'z_2']:
        run_data[measurement_name] = run_data[measurement_name].interpolate(method='linear')
    run_data.to_csv(f'../data/features_interpolated_run{run_id}.csv', index=True)
    table_by_uuid[run_id] = run_data


/var/folders/zm/zbtv1rbj3ndg9ctssd4tcm080000gn/T/ipykernel_1401/1967778286.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for run_id, run_data in feat_groups:


In [244]:
for k in table_by_uuid:
    print(k)
    print(table_by_uuid[k].head())

6.176976534744076e+18
       run_uuid                       pddatetime        fx_1       fx_2  \
0  6.176977e+18 2022-11-23 20:40:00.007000+00:00  176.096381        NaN   
1  6.176977e+18 2022-11-23 20:40:00.008000+00:00  176.685607        NaN   
2  6.176977e+18 2022-11-23 20:40:00.010000+00:00  177.274833  50.373027   
3  6.176977e+18 2022-11-23 20:40:00.011000+00:00  177.864059  50.502727   
4  6.176977e+18 2022-11-23 20:40:00.017000+00:00  178.453284  50.632427   

         fy_1        fy_2        fz_1       fz_2      x_1       x_2      y_1  \
0  174.268623         NaN -258.179417        NaN      NaN       NaN      NaN   
1  173.786072         NaN -258.501301        NaN      NaN  1438.412      NaN   
2  173.303520 -416.060405 -258.823185  80.691381      NaN  1438.412      NaN   
3  172.820969 -415.747946 -259.145069  80.698552  1440.79  1438.412  936.925   
4  172.338417 -415.435487 -259.466953  80.705724  1440.79  1438.412  936.925   

       y_2     z_1  z_2  
0      NaN     NaN  

In [137]:
features[features['run_uuid'] == 7582293080991469568].head()

,run_uuid,time,fx_1,fx_2,fy_1,fy_2,fz_1,fz_2,x_1,x_2,y_1,y_2,z_1,z_2
200151,7.582293e+18,2022-11-23T20:40:00.001Z,-1192.046953,NaN,716.528276,NaN,-1547.340972,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200152,7.582293e+18,2022-11-23T20:40:00.003Z,NaN,-546.669903,NaN,489.207227,NaN,84.484822,NaN,NaN,NaN,NaN,NaN,NaN
200153,7.582293e+18,2022-11-23T20:40:00.008Z,NaN,NaN,NaN,NaN,NaN,NaN,2862.302,NaN,995.86,NaN,-48.565,NaN
200154,7.582293e+18,2022-11-23T20:40:00.009Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3050.773,NaN,1000.769,NaN,-771.632
200155,7.582293e+18,2022-11-23T20:40:00.011Z,-1194.303789,NaN,714.488726,NaN,-1547.280913,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.3 Engineered Features

In [10]:

# assume total values are cumulative
# create a series with column names: vx_1, vy_1, vz_1, vx_2, vy_2, vz_2, ax_1, ay_1, az_1, ax_2, ay_2, az_2, v1, v2, a1, a2, f1, f2
rows = []


# x1 = features[['run_uuid', 'time', 'fx_1']].dropna()
# x1['time_obj'] = x1['time'].apply(convert_time)
# x1['fx_1'].diff()
engineered_features_by_uuid = {}
for run_uuid in table_by_uuid:
    run_data = table_by_uuid[run_uuid]
    engr_feat = pd.DataFrame() # columns=['pddatetime', 'vx_1', 'vy_1', 'vz_1', 'vx_2', 'vy_2', 'vz_2', 'ax_1', 'ay_1', 'az_1', 'ax_2', 'ay_2', 'az_2', 'v1', 'v2', 'a1', 'a2', 'f1', 'f2', ''])
    engr_feat['pddatetime'] = run_data['pddatetime']
    engr_feat['dt'] = run_data['pddatetime'].diff().dt.total_seconds() * 1000

    print(run_data['pddatetime'].diff().dt.total_seconds().head())
    engr_feat['vx_1'] = run_data['x_1'].diff() 
    engr_feat['vy_1'] = run_data['y_1'].diff() 
    engr_feat['vz_1'] = run_data['z_1'].diff() 
    engr_feat['vx_2'] = run_data['x_2'].diff() 
    engr_feat['vy_2'] = run_data['y_2'].diff() 
    engr_feat['vz_2'] = run_data['z_2'].diff() 

    engr_feat['ax_1'] = engr_feat['vx_1'].diff() 
    engr_feat['ay_1'] = engr_feat['vy_1'].diff() 
    engr_feat['az_1'] = engr_feat['vz_1'].diff() 
    engr_feat['ax_2'] = engr_feat['vx_2'].diff() 
    engr_feat['ay_2'] = engr_feat['vy_2'].diff() 
    engr_feat['az_2'] = engr_feat['vz_2'].diff() 

    engr_feat['p_1_sum'] = np.sqrt(run_data['x_1']**2 + run_data['y_1']**2 + run_data['z_1']**2)
    engr_feat['p_2_sum'] = np.sqrt(run_data['x_2']**2 + run_data['y_2']**2 + run_data['z_2']**2)
    engr_feat['v_1_sum'] = np.sqrt(engr_feat['vx_1']**2 + engr_feat['vy_1']**2 + engr_feat['vz_1']**2)
    engr_feat['v_2_sum'] = np.sqrt(engr_feat['vx_2']**2 + engr_feat['vy_2']**2 + engr_feat['vz_2']**2)
    engr_feat['a_1_sum'] = np.sqrt(engr_feat['ax_1']**2 + engr_feat['ay_1']**2 + engr_feat['az_1']**2)
    engr_feat['a_2_sum'] = np.sqrt(engr_feat['ax_2']**2 + engr_feat['ay_2']**2 + engr_feat['az_2']**2)
    engr_feat['f_1_sum'] = np.sqrt(run_data['fx_1']**2 + run_data['fy_1']**2 + run_data['fz_1']**2)
    engr_feat['f_2_sum'] = np.sqrt(run_data['fx_2']**2 + run_data['fy_2']**2 + run_data['fz_2']**2)

    engr_feat['p1'] = engr_feat['p_1_sum'].cumsum()
    engr_feat['p2'] = engr_feat['p_2_sum'].cumsum()
    engr_feat['v1'] = engr_feat['v_1_sum'].cumsum()
    engr_feat['v2'] = engr_feat['v_2_sum'].cumsum()
    engr_feat['a1'] = engr_feat['a_1_sum'].cumsum()
    engr_feat['a2'] = engr_feat['a_2_sum'].cumsum()
    engr_feat['f1'] = engr_feat['f_1_sum'].cumsum()
    engr_feat['f2'] = engr_feat['f_2_sum'].cumsum()

    engineered_features_by_uuid[run_uuid] = engr_feat
    engr_feat.to_csv(f'../data/engr_feat_{run_uuid}.csv')
    

0      NaN
1    0.001
2    0.002
3    0.001
4    0.006
Name: pddatetime, dtype: float64
200151      NaN
200152    0.002
200153    0.005
200154    0.001
200155    0.002
Name: pddatetime, dtype: float64
392042      NaN
392043    0.003
392044    0.009
392045    0.003
392046    0.009
Name: pddatetime, dtype: float64
481783     NaN
481784    0.01
481785    0.01
481786    0.01
481787    0.01
Name: pddatetime, dtype: float64


In [167]:
print(x1.loc[489544,'time_obj'])
print(x1.loc[489545,'time_obj'])
print(x1.loc[489546,'time_obj'])

2022-11-23 20:41:17.630000
2022-11-23 20:41:17.600000
2022-11-23 20:41:17


In [168]:
print(x1.loc[489545,'time_obj'] - x1.loc[489544,'time_obj'])

-1 days +23:59:59.970000


In [157]:
d = datetime.strptime('2022-11-23T20:40:00.007Z', '%Y-%m-%dT%H:%M:%S.%fZ')


